In [1]:
from agents.agent import run_agent

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="pydantic")

Things to remember to talk about
- prompt caching
- function calling
- reasonign models - thought tokens - need to be passed back - thinking traces need to be passed back
- LLM agent abstractions - less hackable?
- litellm
- advanced shit - https://docs.litellm.ai/blog/anthropic_advanced_features#tool-search
- https://www.anthropic.com/engineering/advanced-tool-use
- dont want a lot of tools
- progressive disclosure - tool search 
- https://docs.litellm.ai/blog/gemini_3_flash
- https://docs.litellm.ai/blog/anthropic_advanced_features
- https://github.com/openai/openai-agents-python
- https://www.anthropic.com/engineering/multi-agent-research-system
- context - tools blow it up

In [2]:
run_agent([{'role': 'user', 'content': 'What is the capital of France?'}])

[{'content': 'What is the capital of France?', 'role': 'user'}]


'The capital of France is Paris.'

In [3]:
from litellm import completion
import os
from dotenv import load_dotenv
load_dotenv()

response = completion(
    model="gemini/gemini-3-flash-preview",
    messages=[{"role": "user", "content": "Hello!"}],
    reasoning_effort="low"
)

print(response.choices[0].message.content)

Hello! How can I help you today?


In [4]:
import litellm
litellm.supports_function_calling(model="claude-opus-4-5-20251101")

True

In [15]:
from agents.tools import TOOLS, TOOL_FUNCTIONS
from pprint import pprint
import json


def run_agent():
    # Step 1: Initial request
    messages = [{'role': 'user', 'content': 'What is the capital of France? What is 4847847*2409224 What is the weather in Tokyo? Use python to calc todays date. Then take that date and plug back into python and take sqrt.'}]

    while True:
        pprint(messages)
        # Call the LLM
        response = litellm.completion(
            model="gpt-5.2", # "gemini/gemini-3-flash-preview", #claude-opus-4-5-20251101
            messages=messages,
            tools=TOOLS,
            reasoning_effort="low"
        )

        # Append assistant message (thought signatures automatically preserved)
        message = response.choices[0].message

        # If no tool calls, we're done - return the response
        if not message.tool_calls:
            return message.content

        # Otherwise, process tool calls
        messages.append(message)  # Add assistant message with tool calls

        # Execute tool and append result
        for tool_call in message.tool_calls:
            name = tool_call.function.name
            args = json.loads(tool_call.function.arguments)

            print(f"Calling tool: {name}({args})")

            # Execute the tool
            result = TOOL_FUNCTIONS[name](**args)

            # Add tool result to messages
            messages.append(
                {
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": result,
                }
            )

In [16]:
run_agent()

[{'content': 'What is the capital of France? What is 4847847*2409224 What is '
             'the weather in Tokyo? Use python to calc todays date. Then take '
             'that date and plug back into python and take sqrt.',
  'role': 'user'}]
Calling tool: get_weather({'city': 'Tokyo'})
Calling tool: run_code({'expression': '4847847*2409224'})
Calling tool: run_code({'expression': 'import datetime; datetime.date.today().isoformat()'})
[{'content': 'What is the capital of France? What is 4847847*2409224 What is '
             'the weather in Tokyo? Use python to calc todays date. Then take '
             'that date and plug back into python and take sqrt.',
  'role': 'user'},
 Message(content=None, role='assistant', tool_calls=[ChatCompletionMessageToolCall(function=Function(arguments='{"city": "Tokyo"}', name='get_weather'), id='call_SSrNMoeHd81VWpHUnNn6maY5', type='function'), ChatCompletionMessageToolCall(function=Function(arguments='{"expression": "4847847*2409224"}', name='run_co

'- Capital of France: **Paris**\n- \\(4847847 \\times 2409224 = \\) **11679549340728**\n- Weather in Tokyo: **72°F and sunny**\n\nPython-calculated “today’s date”:\n- Using Python `date.today().toordinal()` (a numeric form of today’s date): **739626**\n\nThen sqrt in Python:\n- \\(\\sqrt{739626} \\approx\\) **860.0151161462221**'

In [11]:
print('- **Capital of France:** Paris  \n- **4847847 × 2409224:** **11,679,549,340,728**  \n- **Weather in Tokyo:** **72°F and sunny**  \n\n**Python “today’s date”, then sqrt using that date**\n- Today’s date (ISO): **2026-01-10**\n\nIf we “plug that date back into python” in a numeric way, a standard approach is to convert it to an **ordinal day number** (what Python’s `date.toordinal()` uses).  \nFor **2026-01-10**, the ordinal is **739626**, and:\n\n- **sqrt(739626) ≈ 859.899412**')

- **Capital of France:** Paris  
- **4847847 × 2409224:** **11,679,549,340,728**  
- **Weather in Tokyo:** **72°F and sunny**  

**Python “today’s date”, then sqrt using that date**
- Today’s date (ISO): **2026-01-10**

If we “plug that date back into python” in a numeric way, a standard approach is to convert it to an **ordinal day number** (what Python’s `date.toordinal()` uses).  
For **2026-01-10**, the ordinal is **739626**, and:

- **sqrt(739626) ≈ 859.899412**
